# Train search engin

In [1]:
! pip install -U sentence-transformers

Requirement already up-to-date: sentence-transformers in c:\users\evangelia.zve\anaconda3\lib\site-packages (2.2.2)


In [2]:
! pip install wordcloud
! pip install openpyxl
! pip install torch
! pip install transformers
! pip install pandas
! pip install faiss-cpu
! pip install folium
! pip install streamlit

In [3]:
! pip install s3fs

In [4]:
import numpy as np

def vector_search(query, model, index, num_results=10):
    """Tranforms query to vector using a pretrained, sentence-level 
    DistilBERT model and finds similar vectors using FAISS.
    Args:
        query (str): User query that should be more than a sentence long.
        model (sentence_transformers.SentenceTransformer.SentenceTransformer)
        index (`numpy.ndarray`): FAISS index that needs to be deserialized.
        num_results (int): Number of results to return.
    Returns:
        D (:obj:`numpy.array` of `float`): Distance between results and query.
        I (:obj:`numpy.array` of `int`): Paper ID of the results.
    
    """
    vector = model.encode(list(query))
    D, I = index.search(np.array(vector).astype("float32"), k=num_results)
    return D, I

def id2details(df, I, column):
    """Returns the paper titles based on the paper index."""
    return [list(df[df.id == idx][column]) for idx in I[0]]

In [5]:
# Used to import data from S3.
import pandas as pd
import s3fs

# Used to create the dense document vectors.
import torch
from sentence_transformers import SentenceTransformer

# Used to create and store the Faiss index.
import faiss
import numpy as np
import pickle
from pathlib import Path

In [20]:
df_objets = pd.read_excel('C:/Users/evangelia.zve/Documents/acidata/fonctions normees/liste_fonction_fr_en.xlsx')

In [21]:
df_objets.head()

,occupation
0,directeur technique/directrice technique
1,opérateur de tréfilerie/opératrice de tréfilerie
2,contrôleur qualité des instruments de précisio...
3,technicien de la sécurité du trafic aérien/tec...
4,responsable yield manager


In [22]:
objects_campaign_pd_processed_text = df_objets["occupation"].dropna()
objects_campaign_pd_processed_text = objects_campaign_pd_processed_text.drop_duplicates()
objects_campaign_pd_processed_text = list(objects_campaign_pd_processed_text)

In [23]:
objects_campaign_pd_processed_text = [s.lower() for s in objects_campaign_pd_processed_text]

In [24]:
# Instantiate the sentence-level DistilBERT
model = SentenceTransformer("distilbert-base-uncased")

No sentence-transformers model found with name C:\Users\evangelia.zve/.cache\torch\sentence_transformers\distilbert-base-uncased. Creating a new one with MEAN pooling.


Some weights of the model checkpoint at C:\Users\evangelia.zve/.cache\torch\sentence_transformers\distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
# Convert job titles to vectors
embeddings = model.encode(objects_campaign_pd_processed_text, show_progress_bar=True)

In [26]:
print(f'Shape of the vectorised occupations: {embeddings.shape}')

Shape of the vectorised occupations: (5875, 768)


In [28]:
# Step 1: Change data type
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")
# Step 2: Instantiate the index
index = faiss.IndexFlatL2(embeddings.shape[1])
#index = faiss.IndexFlatIP(embeddings.shape[1])
# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)
# Step 4: Add vectors and their IDs
index.add_with_ids(embeddings,  np.array(range(0,5875)))
print(f"Number of vectors in the Faiss index: {index.ntotal}")

Number of vectors in the Faiss index: 5875


In [30]:
# Serialise index and store it as a pickle
with open(f"C:/Users/evangelia.zve/Documents/faiss_index_esco.pickle", "wb") as h:
    pickle.dump(faiss.serialize_index(index), h)

In [31]:
with open(f"C:/Users/evangelia.zve/Documents/fonctions_list.pickle", "wb") as h:   # Unpickling
    pickle.dump(objects_campaign_pd_processed_text, h)